In [1]:
%pip -q install google-genai

In [2]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [3]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [4]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

# Perguntar pro modelo quando é a próxima imersão de IA ###############################################
resposta = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {resposta.text}"))

Resposta:
 A Alura não tem uma data fixa para a Imersão IA com Google Gemini. A melhor forma de saber quando será a próxima edição é:

*   **Acompanhar as redes sociais da Alura:** Fique de olho nos perfis da Alura no [Instagram](https://www.instagram.com/aluraonline/) e [LinkedIn](https://www.linkedin.com/school/aluraonline/). Eles costumam anunciar novas edições por lá.
*   **Verificar a página de Imersões da Alura:** Acesse a página de Imersões no site da Alura e procure por informações sobre a Imersão IA com Google Gemini.
*   **Assinar a newsletter da Alura:** Ao assinar a newsletter, você recebe informações sobre os próximos eventos e cursos da Alura.

In [5]:
# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
    config={"tools": [{"google_search": {}}]}
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 A Alura e o Google realizaram uma Imersão IA com Google Gemini em maio de 2024. A inscrição para a Imersão IA foi feita exclusivamente pelo site até 11 de maio de 2025.

Como as inscrições se encerraram em 11 de maio de 2025, não há informações disponíveis sobre a próxima Imersão IA com Google Gemini da Alura.

In [6]:
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Busca realizada: ['Próxima Imersão IA com Google Gemini Alura']
Páginas utilizadas na resposta: youtube.com, alura.com.br



In [7]:
# Instalar Framework de agentes do Google ################################################
!pip install -q google-adk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.1/218.1 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 335.7/335.7 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.5/229.5 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.8/103.8 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [8]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [9]:
# Cria um serviço de sessão em memória
session_service = InMemorySessionService()

# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
async def call_agent(agent: Agent, message_text: str) -> str:
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = await session_service.create_session(app_name=agent.name, user_id="user1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    async for event in runner.run_async(user_id="user1", session_id=session.id, new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [10]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [11]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################
async def agente_buscador(topico, data_de_hoje):

    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um assistente de pesquisa. A sua tarefa é usar a ferramenta de busca do google (google_search)
        para recuperar as últimas notícias de lançamentos muito relevantes sobre o tópico abaixo.
        Foque em no máximo 5 lançamentos relevantes, com base na quantidade e entusiasmo das notícias sobre ele.
        Se um tema tiver poucas notícias ou reações entusiasmadas, é possível que ele não seja tão relevante assim
        e pode ser substituído por outro que tenha mais.
        Esses lançamentos relevantes devem ser atuais, de no máximo um mês antes da data de hoje.
        """,
        description="Agente que busca informações no Google",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"

    lancamentos = await call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos

In [20]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################
async def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Você é um planejador de conteúdo, especialista em redes sociais. Com base na lista de
        lançamentos mais recentes e relevantes buscador, você deve:
        usar a ferramenta de busca do Google (google_search) para criar um plano sobre
        quais são os pontos mais relevantes que poderíamos abordar em um post sobre
        cada um deles. Você também pode usar o (google_search) para encontrar mais
        informações sobre os temas e aprofundar.
        Ao final, você irá escolher o tema mais relevante entre eles com base nas suas pesquisas
        e retornar esse tema, seus pontos mais relevantes, e um plano com os assuntos
        a serem abordados no post que será escrito posteriormente.
        """,
        description="Agente que planeja posts",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = await call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [34]:
######################################
# --- Agente 3: Redator do Post --- #
######################################
async def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Redator Criativo especializado em criar posts virais para redes sociais.
            Você escreve posts para a empresa Alura, a maior escola online de tecnologia do Brasil.
            Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um rascunho de post para Instagram sobre o tema indicado.
            O post deve ser engajador, informativo, com linguagem simples e incluir 2 a 4 hashtags no final.
            """,
        description="Agente redator de posts engajadores para Instagram"
     )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho = await call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [35]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
async def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em posts para redes sociais, com foco no Instagram.
            Por ter um público jovem, entre 18 e 30 anos, use um tom de escrita adequado.
            Revise o rascunho de post de Instagram abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de post para redes sociais."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = await call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [36]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
    print("Você esqueceu de digitar o tópico!")
else:
    print(f"Maravilha! Vamos então criar o post sobre novidades em {topico}")

    lancamentos_buscados = await agente_buscador(topico, data_de_hoje)
    print("\n--- 📝 Resultado do Agente 1 (Buscador) ---\n")
    display(to_markdown(lancamentos_buscados))
    print("--------------------------------------------------------------")

    plano_de_post = await agente_planejador(topico, lancamentos_buscados)
    print("\n--- 📝 Resultado do Agente 2 (Planejador) ---\n")
    display(to_markdown(plano_de_post))
    print("--------------------------------------------------------------")

    rascunho_de_post = await agente_redator(topico, plano_de_post)
    print("\n--- 📝 Resultado do Agente 3 (Redator) ---\n")
    display(to_markdown(rascunho_de_post))
    print("--------------------------------------------------------------")

    post_final = await agente_revisor(topico, rascunho_de_post)
    print("\n--- 📝 Resultado do Agente 4 (Revisor) ---\n")
    display(to_markdown(post_final))
    print("--------------------------------------------------------------")

🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀
❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: mudança de carreira em t.i.
Maravilha! Vamos então criar o post sobre novidades em mudança de carreira em t.i.

--- 📝 Resultado do Agente 1 (Buscador) ---



> Aqui estão algumas das notícias mais relevantes sobre mudanças de carreira em TI de aproximadamente um mês antes de hoje:
> 
> 
> Com base nas notícias mais recentes sobre mudanças de carreira em TI, aqui estão alguns lançamentos e tendências relevantes:
> 
> 1.  **Alta Demanda e Oportunidades:** O setor de TI continua a ser um dos mais aquecidos, com alta demanda por profissionais qualificados. A transformação digital acelerada pela pandemia impulsionou a necessidade de especialistas em tecnologia, criando um cenário favorável para quem busca uma transição de carreira.
> 2.  **Profissões em Destaque:** Carreiras ligadas à análise de dados, cibersegurança e inteligência artificial (IA) estão em alta. Cargos como gerente de TI, desenvolvedor back-end, coordenador de segurança da informação e gerente de projetos estão entre os mais procurados.
> 3.  **Bootcamps como Porta de Entrada:** Bootcamps de tecnologia são uma opção popular para quem deseja mudar de carreira rapidamente. Esses cursos oferecem treinamento prático e focado nas habilidades exigidas pelo mercado. A DIO (Digital Innovation One), em colaboração com a Microsoft, oferece bootcamps com foco em computação em nuvem com Azure, ideais para iniciantes e profissionais em transição.
> 4.  **Habilidades Essenciais:** Além das habilidades técnicas, as *soft skills* são cada vez mais importantes. Pensamento crítico, comunicação clara, visão de negócios e capacidade de tomar decisões baseadas em dados são características valorizadas pelas empresas.
> 5.  **Salários Atrativos:** O setor de tecnologia oferece salários competitivos, com médias salariais que podem variar de R$ 4.017,00 para desenvolvedores júnior a R$ 15 mil - R$ 20 mil para posições mais especializadas, como BI Manager e Especialista em Segurança da Informação. Mesmo sem formação em TI, especialistas em IA podem ter salários iniciais em torno de R$ 7 mil, podendo chegar a R$ 35 mil em cargos de gestão.
> 6.  **Empresas Contratando:** Diversas empresas estão com vagas abertas e oferecem programas de capacitação para atrair novos talentos. Microsoft, Vivo e Porto são algumas das empresas que oferecem programas de estágio e cursos gratuitos em tecnologia. A consultoria Labsit também está contratando profissionais sem diploma em TI, priorizando o perfil e conhecimento em tecnologia.
> 
> Essas tendências indicam que a área de TI continua a ser promissora para quem busca uma mudança de carreira, oferecendo diversas oportunidades e salários atrativos.


--------------------------------------------------------------

--- 📝 Resultado do Agente 2 (Planejador) ---



> Okay, com base nas informações fornecidas, vou criar um plano para um post sobre mudança de carreira em TI. Primeiro, farei algumas pesquisas adicionais para complementar as informações que você me forneceu.
> 
> 
> Com base nas informações fornecidas e nas pesquisas adicionais, aqui está um plano para um post sobre mudança de carreira em TI:
> 
> **Tema Escolhido:** Mudança de Carreira em TI: Oportunidades, Desafios e Como Começar
> 
> **Por que este tema?**
> 
> *   **Relevância:** O setor de TI continua aquecido, oferecendo diversas oportunidades para quem busca uma transição de carreira.
> *   **Interesse:** Muitas pessoas consideram migrar para TI em busca de melhores salários, flexibilidade e a possibilidade de trabalhar com tecnologias inovadoras.
> *   **Abrangência:** Permite abordar tanto os aspectos positivos quanto os desafios da transição, fornecendo um guia prático para os interessados.
> 
> **Pontos Relevantes a Abordar:**
> 
> 1.  **Cenário Atual do Mercado de TI:**
>     *   Demanda crescente por profissionais qualificados.
>     *   Áreas de destaque: análise de dados, cibersegurança, IA, desenvolvimento de software.
>     *   Impacto da transformação digital e da pandemia.
> 2.  **Motivações para a Transição:**
>     *   Salários atrativos e benefícios.
>     *   Flexibilidade e possibilidade de trabalho remoto.
>     *   Oportunidade de trabalhar com tecnologias inovadoras.
>     *   Busca por crescimento profissional e novos desafios.
> 3.  **Como Iniciar a Carreira em TI:**
>     *   **Educação e Capacitação:**
>         *   Cursos técnicos, graduações e certificações.
>         *   Bootcamps como alternativa rápida e prática.
>         *   Plataformas online com cursos acessíveis (Coursera, Udemy, DIO).
>         *   Certificações de nível básico (CompTIA A+).
>     *   **Desenvolvimento de Habilidades:**
>         *   *Hard skills:* Linguagens de programação (Python, JavaScript), banco de dados, sistemas operacionais.
>         *   *Soft skills:* Pensamento crítico, comunicação clara, visão de negócios, capacidade de tomar decisões baseadas em dados, inteligência emocional.
>     *   **Networking e Mentoria:**
>         *   Participar de eventos, workshops e comunidades online.
>         *   Buscar mentoria de profissionais experientes.
>     *   **Construção de Portfólio:**
>         *   Projetos pessoais e contribuições para projetos de código aberto.
>         *   Criação de pequenos aplicativos.
> 4.  **Desafios da Transição:**
>     *   Falta de experiência prévia.
>     *   Necessidade de aprendizado contínuo e adaptação às novas tecnologias.
>     *   Concorrência no mercado de trabalho.
>     *   Dificuldades financeiras durante o período de transição.
> 5.  **Exemplos de Empresas que Contratam e Capacitam:**
>     *   Microsoft, Vivo, Porto.
>     *   Consultorias como Labsit.
>     *   Empresas que oferecem programas de estágio e cursos gratuitos.
> 6.  **Salários em TI:**
>     *   Média salarial para desenvolvedores júnior: R$ 4.017,00.
>     *   Salários para posições especializadas (BI Manager, Especialista em Segurança da Informação): R$ 15 mil - R$ 20 mil.
>     *   Salários iniciais para especialistas em IA: R$ 7 mil (podendo chegar a R$ 35 mil em cargos de gestão).
>     *   Salários podem chegar até R$ 51 mil dependendo do cargo, experiência e empresa.
>     *   Analista de TI federal: R$ 11.150,80 (inicial) a R$ 21.613,10 (em 2026).
> 7.  **Soft Skills Essenciais:**
>     *   Comunicação.
>     *   Trabalho em equipe.
>     *   Inteligência emocional.
>     *   Adaptabilidade.
>     *   Resiliência.
>     *   Pensamento crítico.
> 
> **Plano de Assuntos para o Post:**
> 
> 1.  **Título:** "Guia Completo: Como Mudar para uma Carreira em TI em 2025" (ou similar)
> 2.  **Introdução:**
>     *   Contextualizar o cenário promissor do mercado de TI.
>     *   Apresentar o objetivo do guia: fornecer informações e dicas práticas para quem deseja mudar de carreira.
> 3.  **Por que TI?**
>     *   Listar os principais motivos que levam as pessoas a buscar uma carreira em TI (salários, flexibilidade, inovação).
> 4.  **Áreas em Destaque:**
>     *   Apresentar as áreas de TI com maior demanda e potencial de crescimento (desenvolvimento, dados, segurança, IA).
> 5.  **Como Começar:**
>     *   Detalhar os passos para iniciar a carreira em TI, abordando educação, habilidades, networking e portfólio.
> 6.  **Superando os Desafios:**
>     *   Discutir os principais desafios da transição e como superá-los (falta de experiência, aprendizado contínuo, concorrência).
> 7.  **Empresas que Acreditam em Novos Talentos:**
>     *   Listar empresas que oferecem programas de capacitação, estágio e oportunidades para iniciantes.
> 8.  **Salários e Perspectivas:**
>     *   Apresentar uma visão geral dos salários em diferentes áreas e níveis de experiência.
> 9.  **Desenvolvendo as Soft Skills:**
>     *   Enfatizar a importância das *soft skills* e como desenvolvê-las (cursos, treinamentos, prática).
> 10. **Histórias de Sucesso:**
>     *   Compartilhar histórias inspiradoras de pessoas que fizeram a transição para TI e alcançaram sucesso.
> 11. **Conclusão:**
>     *   Reforçar que a transição é possível com planejamento, dedicação e as ferramentas certas.
>     *   Incentivar os leitores a darem o primeiro passo e buscar mais informações.
> 
> Este plano deve fornecer um bom ponto de partida para criar um post informativo e útil para quem está considerando uma mudança de carreira para a área de TI.
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 3 (Redator) ---



> 🚀 Quer mudar de carreira e #partiu ser tech em 2025? 🚀
> 
> Se liga nesse guia COMPLETO pra você dar um UP na sua vida profissional e entrar no mundo da tecnologia! 🤩
> 
> 💰 Por que TI? Salários que brilham, flexibilidade e a chance de trabalhar com INOVAÇÃO! ✨
> 
> Áreas que bombam: desenvolvimento, dados, segurança e IA tão te esperando! 💻
> 
> 🤔 Como começar? Cursos na Alura (spoiler 😉), habilidades afiadas, networking e um portfólio DE RESPEITO! 🚀
> 
> 💪 Desafios? Medo de não ter experiência? A gente te ajuda a superar!
> 
> E as empresas? Microsoft, Vivo e Porto te dão a mão pra começar! 🤝
> 
> 💰 Salários? De R$ 4.017 (júnior) até R$ 51 MIL (experiente)! 🤑
> 
> Soft skills? Comunicação, trabalho em equipe e muita INTELIGÊNCIA EMOCIONAL! 😉
> 
> 👉 Tá esperando o quê? O futuro te espera na TI! 🚀 #CarreiraTech #Alura #MudeParaTI #Desenvolvimento
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 4 (Revisor) ---



> O rascunho está ótimo, mas podemos deixá-lo ainda mais atraente para o público do Instagram com algumas pequenas alterações:
> 
> *   **Emojis:** Usar emojis é ótimo, mas podemos otimizar para que cada um reforce a mensagem e adicione um toque visual mais interessante.
> *   **Chamada para Ação (CTA):** A CTA final está boa, mas podemos adicionar um senso de urgência ou curiosidade para aumentar o engajamento.
> *   **Hashtags:** As hashtags estão ok, mas podemos adicionar algumas mais específicas e populares para aumentar o alcance.
> 
> **Sugestões de Melhoria:**
> 
> 🚀 Quer virar tech em 2025 e bombar na carreira? 🚀
> 
> Se liga nesse guia COMPLETO pra você dar um UP na sua vida profissional e finalmente entrar no mundo da tecnologia! 🤩
> 
> 💰 Por que TI? 👀 Salários ATRAENTES, flexibilidade e a chance de trabalhar com INOVAÇÃO que muda o mundo! ✨
> 
> As áreas que mais bombam: desenvolvimento de software, análise de dados, segurança cibernética e Inteligência Artificial te esperam! 💻 Escolha a sua!
> 
> 🤔 Como começar? Cursos na Alura (já sabe, né? 😉), habilidades técnicas afiadas, networking e um portfólio de respeito! 🚀
> 
> 💪 Medo de não ter experiência? Relaxa! A gente te ajuda a superar esse e todos os desafios! 😉
> 
> E as empresas? Gigantes como Microsoft, Vivo e Porto estão de portas abertas para quem está começando! 🤝
> 
> 💰 Salários? De R$ 4.017 (júnior) até R$ 51 MIL (experiente)! 🤑 Se prepare para realizar seus sonhos!
> 
> Soft skills? Comunicação clara, trabalho em equipe e muita inteligência emocional são ESSENCIAIS! 😉
> 
> 👉 Tá esperando o quê pra começar? Clica no link da bio e descubra como dar o primeiro passo! 🚀 #CarreiraTech #Alura #MudeParaTI #Desenvolvimento #TechBrasil #FuturoNaTI
> 


--------------------------------------------------------------
